In [1]:
print("Hello")

Hello


In [4]:

! pip install openai -q
! pip install python-dotenv -q

In [1]:
! pip install openai pydub -q

In [2]:
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv 


load_dotenv()
client = OpenAI()


speech_file_path = "harvard.wav"
response = client.audio.speech.create(
  # model="tts-1",
  model="tts-1-hd",
  # voice="alloy",
  voice = "echo",
  # voice = "fable",
  # voice = "onyx",
  # voice = "nova",
  # voice = "shimmer",
  input="Your time is limited, so don’t waste it living someone else’s life. Don’t be trapped by dogma — which is living with the results of other people’s thinking. Don’t let the noise of others’ opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition. They somehow already know what you truly want to become. Everything else is secondary...Stay Hungry. Stay Foolish."
)


response.stream_to_file("harvard.mp3")


/var/folders/_1/hfb_ghvx10q00n235p7x9s4m0000gn/T/ipykernel_30206/3683541761.py:24: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("harvard.mp3")


In [11]:
speech_file_path = "different_language.mp3"

response = client.audio.speech.create(
    model="tts-1",
    voice="onyx",
    input="जिस चीज को आप चाहते हैं, उसमें असफल होना, जिस चीज को आप नहीं चाहते उसमें सफल होने से बेहतर है।"
)

response.stream_to_file(speech_file_path)

/var/folders/_1/hfb_ghvx10q00n235p7x9s4m0000gn/T/ipykernel_4851/2205648268.py:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [12]:
audio_file= open("/Users/vinayak/AI/projects/voice-assistant-bot/harvard.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1",
  response_format="text", # Default output format is json,if want in json format, just comment out response format
  file=audio_file
)
transcript

"Your time is limited, so don't waste it living someone else's life. Don't be trapped by dogma, which is living with the results of other people's thinking. Don't let the noise of others' opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition. They somehow already know what you truly want to become. Everything else is secondary. Stay hungry, stay foolish.\n"

In [3]:
audio_file= open("/Users/vinayak/AI/projects/voice-assistant-bot/different_language.mp3", "rb")

transcript = client.audio.translations.create(
  model="whisper-1",
  response_format = "text",
  file=audio_file
)

print("Translated Transcript: ", transcript)
print("\n")

original_transcript = client.audio.transcriptions.create(
  model="whisper-1",
  response_format="text",
  file=audio_file
)

print("Original Transcript: ", original_transcript)

Translated Transcript:  It is better to be unsuccessful in what you want than to be successful in what you don't want.



Original Transcript:  जिस चीज़ को आप चाहते हैं, उसमें असफल होना। जिस चीज़ को आप नहीं चाहते, उसमें सफल होने से बहतर है।



In [3]:
import os
from pydub import AudioSegment
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

# Define the text for each speaker
text_speaker_1 = "Your time is limited, so don’t waste it living someone else’s life. Don’t be trapped by dogma — which is living with the results of other people’s thinking."
text_speaker_2 = "Don’t let the noise of others’ opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition. They somehow already know what you truly want to become. Everything else is secondary...Stay Hungry. Stay Foolish."

# Generate speech for each speaker
response_speaker_1 = client.audio.speech.create(
    model="tts-1-hd",
    voice="echo",
    input=text_speaker_1
)

response_speaker_2 = client.audio.speech.create(
    model="tts-1-hd",
    voice="fable",
    input=text_speaker_2
)

# Save the audio files
audio_file_path_1 = "speaker_1.wav"
audio_file_path_2 = "speaker_2.wav"

with open(audio_file_path_1, "wb") as f:
    f.write(response_speaker_1.content)

with open(audio_file_path_2, "wb") as f:
    f.write(response_speaker_2.content)

# Combine the audio files using pydub
audio1 = AudioSegment.from_wav(audio_file_path_1)
audio2 = AudioSegment.from_wav(audio_file_path_2)

combined_audio = audio1 + AudioSegment.silent(duration=500) + audio2

# Export the combined audio to a file
combined_audio.export("podcast.wav", format="wav")

print("Podcast audio saved as podcast.wav")


/Users/vinayak/AI/projects/voice-assistant-bot/voice/lib/python3.10/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe'

In [4]:
import whisper 

def transcribe(audio_file: str, model_name: str, device: str = "cpu"):
    """
    Transcribe an audio file using a speech-to-text model.

    Args:
        audio_file: Path to the audio file to transcribe.
        model_name: Name of the model to use for transcription.
        device: The device to use for inference (e.g., "cpu" or "cuda").

    Returns:
        A dictionary representing the transcript, including the segments, the language code, and the duration of the audio file.
    """
    model = whisper.load_model(model_name, device)
    result = model.transcribe(audio_file)

    language_code = result["language"]
    return {
        "segments": result["segments"],
        "language_code": language_code,
    }


transcribe(audio_file = "/Users/vinayak/AI/projects/voice-assistant-bot/harvard.mp3" , model_name="base")

/Users/vinayak/AI/projects/voice-assistant-bot/voice/lib/python3.10/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'

In [1]:
! pip install torch torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
! pip install git+https://github.com/pyannote/pyannote-audio


  Cloning https://github.com/pyannote/pyannote-audio to /private/var/folders/_1/hfb_ghvx10q00n235p7x9s4m0000gn/T/pip-req-build-sp87ft6r
  Running command git clone --filter=blob:none --quiet https://github.com/pyannote/pyannote-audio /private/var/folders/_1/hfb_ghvx10q00n235p7x9s4m0000gn/T/pip-req-build-sp87ft6r
  Resolved https://github.com/pyannote/pyannote-audio to commit 286ea1a4e34e2dd7d7926f590e402dac1e17494b
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Using cached torch-2.3.1-cp310-none-macosx_11_0_arm64.whl.metadata (26 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 2.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.5 MB/s eta 0:00:0000:0100:01m
Using cached torch-2.3.1-cp310-none-macosx_11_0_arm64.whl (61.0 MB)
  Created wheel for pyannote.audio: filename=pyannote.audio-3.3.1-py2.py3-none-any.whl size=898660 sha256=046e77f45ca71dcda7fdd52fb97e5fed06c9da815854b83168e

In [ ]:
from subprocess import CalledProcessError, run
from pyannote.audio import Pipeline
import os